# RAiDER in Python

RAiDER can be called and used from with Python. 

In [1]:
# at the most basic level, can "import RAiDER"
import RAiDER

In [2]:
# we'll need some other basic libraries
import datetime
import pyproj
import xarray

In [3]:
# to actually do something, we'll need more
# in order: 

# 1. basic data types and input parameters
from RAiDER.llreader import BoundingBox, StationFile
from RAiDER.losreader import Zenith, Conventional, Raytracing

# 2. Weather model access
from RAiDER.models.hrrr import HRRR
from RAiDER.processWM import prepareWeatherModel

# 3. Delay calculation
from RAiDER.delay import tropo_delay

In [4]:
# First define datetime, and AOI
datetime = datetime.datetime(2020,1,1, 12, 0, 0)

# bounding box is given in SNWE format
aoi = BoundingBox([32, 33, -75, -74])

In [5]:
# For the first example, we'll do a zenith calculation at the weather model grid nodes
weather_model = HRRR()
prepareWeatherModel(weather_model, datetime, ll_bounds=aoi.bounds(), makePlots=True)

Extent of the weather model is (xmin, ymin, xmax, ymax):-75.58, 31.89, -73.40, 33.11
Extent of the input is (xmin, ymin, xmax, ymax): -75.00, 32.00, -74.00, 33.00


'/Users/jlmd9g/software/RAiDER-docs/notebooks/RAiDER_tutorial/weather_files/HRRR_2020_01_01_T12_00_00_32N_34N_76W_74W.nc'

You can look at the PDF files generated to see slices of the weather model variables at different heights. 

We can also load the weather model using xarray:

In [6]:
ds = xarray.load_dataset('weather_files/HRRR_2020_01_01_T12_00_00_32N_34N_76W_74W.nc')

In [7]:
ds

<xarray.Dataset>
Dimensions:      (x: 56, y: 62, z: 145)
Coordinates:
  * x            (x) float64 2.063e+06 2.066e+06 ... 2.225e+06 2.228e+06
  * y            (y) float64 -4.893e+05 -4.863e+05 ... -3.093e+05 -3.063e+05
  * z            (z) float32 -500.0 -300.0 -200.0 ... 7.458e+04 8.03e+04
Data variables:
    CRS          |S1 b''
    latitude     (z, y, x) float64 -3.403e+38 -3.403e+38 ... -3.403e+38
    longitude    (z, y, x) float64 -3.403e+38 -3.403e+38 ... -3.403e+38
    t            (z, y, x) float32 293.3 293.4 293.4 293.4 ... 206.9 206.9 206.9
    p            (z, y, x) float32 1.012e+05 1.012e+05 ... 5.339e+03 5.34e+03
    e            (z, y, x) float32 1.287e+03 1.291e+03 ... 0.0193 0.01931
    wet          (z, y, x) float32 57.11 57.3 57.58 ... 0.001712 0.001713
    hydro        (z, y, x) float32 267.7 267.7 267.7 267.7 ... 20.03 20.03 20.03
    wet_total    (z, y, x) float32 0.1146 0.1149 0.1154 0.1159 ... 0.0 0.0 0.0
    hydro_total  (z, y, x) float32 3.532 3.532 3.532 3.532 ... 0.0 0.0 0.0 0.0
Attributes:
    Conventions:   CF-1.6
    datetime:      2020_01_01T12_00_00
    date_created:  2022_12_07T16_00_13
    title:         Weather model data and delay calculations

"wet_total" and "hydro_total" are the zenith delays (ZTD) at the weather model grid nodes. 

We can now run the delay calculation using various input query points

In [9]:
los = Zenith()

In [10]:
ds = tropo_delay(datetime, 'weather_files/HRRR_2020_01_01_T12_00_00_32N_34N_76W_74W.nc', aoi, los)

In [11]:
ds

(<xarray.Dataset>
 Dimensions:          (z: 145, y: 62, x: 56)
 Coordinates:
   * x                (x) float64 2.063e+06 2.066e+06 ... 2.225e+06 2.228e+06
   * y                (y) float64 -4.893e+05 -4.863e+05 ... -3.093e+05 -3.063e+05
   * z                (z) float32 -500.0 -300.0 -200.0 ... 7.458e+04 8.03e+04
 Data variables:
     wet              (z, y, x) float64 nan nan nan nan nan ... nan nan nan nan
     hydro            (z, y, x) float64 nan nan nan nan nan ... nan nan nan nan
     cube_projection  int64 0
 Attributes:
     Conventions:     CF-1.7
     title:           RAiDER geo cube
     source:          HRRR_2020_01_01_T12_00_00_32N_34N_76W_74W.nc
     history:         2022-12-08 22:06:18.044046 RAiDER
     description:     RAiDER geo cube - zenith
     reference_time:  2020-01-01 12:00:00,
 None)